## Итерационные методы решения СЛАУ (метод простой итерации, метод Зейделя).

In [1]:
from numpy import zeros, array, dot, ones
from numpy.linalg import norm, solve
from prettytable import PrettyTable

Метод простой итерации

In [2]:
def find_B_C(A, b):  
    n = A.shape[0]
    B = array(zeros((n, n)))
    C = array(zeros(b.shape[0]))
    for i in range(n):
        for j in range(n):
            if i != j:
                B[i][j] = - A[i][j] / A[i][i]
            C[i] = b[i] / A[i][i]
    return B, C

In [3]:
def simple_iteration_method(B, C, x, eps):
    iter = 1
    new_x = dot(B,x) + C
    while norm(new_x - x) > eps and iter < 500:
        x = new_x
        new_x = dot(B, x) + C
        iter += 1
    return x, iter

Метод Зейделя

In [8]:
def seidel_method(A, b, eps):  
    iter = 0
    n = A.shape[0]
    x = array(zeros((b.shape[0])))
    error = eps + 1
    while error > eps:
        x_new = x.copy()
        for i in range(n):
            sum1 = sum(A[i][j] * x_new[j] for j in range(i))
            sum2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - sum1 - sum2)/A[i][i]
        error = norm(x_new - x)
        iter += 1
        x = x_new
    return x, iter

In [9]:
def hilbert_matrix(n: int):
    matrix = zeros((n, n))
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            matrix[i - 1][j - 1] = 1 / (i + j - 1)
    return matrix

### Тесты

In [10]:
systems_of_equations = []

#7 вариант из методички Пакулиной
A = array([[-402.9, 200.7], [1204.2, -603.6]])
b = array([200, -600])
x = solve(A, b)
systems_of_equations.append({'A': A, 'b': b, 'x': x})

#трехдиагональная матрица с диагональным преобладанием
A = array([[5, 4, 0, 0], [2.75, 3.75, 1, 0], [0, 0.5, 7, 6], [0, 0, 2, 10]])
x = array([1, 1, 1, 1])
b = dot(A, x)
systems_of_equations.append({'A': A, 'b': b, 'x': x})

#матрицы Гильберта размером 2x2, 3x3, 4x4
for size in [2, 3, 4]:
    A = hilbert_matrix(size)
    x = ones(size)
    b = dot(A, x)
    systems_of_equations.append({'A': A, 'x': x, 'b': b})

In [11]:
all_results = []
for system in systems_of_equations:
    B, C = find_B_C(system['A'], system['b'])
    results_iteration_method = PrettyTable()
    results_iteration_method.field_names = ["eps", "кол-во итераций", "norm(x-x*)"]
    results_seidel_method = PrettyTable()
    results_seidel_method.field_names = ["eps", "кол-во итераций", "norm(x-x*)"]
    
    for e in range(-10, 0):
        eps = 10**e
        x_iter, num_of_iter = simple_iteration_method(B, C, system['b'], eps)
        results_iteration_method.add_row([eps, num_of_iter, norm(system['x'] - x_iter)])
        x_iter, num_of_iter = seidel_method(system['A'], system['b'], eps)
        results_seidel_method.add_row([eps, num_of_iter, norm(system['x'] - x_iter)])
    
    all_results.append(results_iteration_method)
    all_results.append(results_seidel_method)

Матрица из методички Пакулиной

In [12]:
print('Метод простой итерации')
print(all_results[0])
print('Метод Зейделя')
print(all_results[1])

Метод простой итерации
+--------+-----------------+------------------+
|  eps   | кол-во итераций |    norm(x-x*)    |
+--------+-----------------+------------------+
| 1e-10  |       500       | 106.096741823152 |
| 1e-09  |       500       | 106.096741823152 |
| 1e-08  |       500       | 106.096741823152 |
| 1e-07  |       500       | 106.096741823152 |
| 1e-06  |       500       | 106.096741823152 |
| 1e-05  |       500       | 106.096741823152 |
| 0.0001 |       500       | 106.096741823152 |
| 0.001  |       500       | 106.096741823152 |
|  0.01  |       500       | 106.096741823152 |
|  0.1   |       500       | 106.096741823152 |
+--------+-----------------+------------------+
Метод Зейделя
+--------+-----------------+------------------------+
|  eps   | кол-во итераций |       norm(x-x*)       |
+--------+-----------------+------------------------+
| 1e-10  |       2822      | 1.598958401223974e-08  |
| 1e-09  |       2452      | 1.5959639948444052e-07 |
| 1e-08  |       2081

Трехдиагональная матрица с диагональным преобладанием

In [13]:
print('Метод простой итерации')
print(all_results[2])
print('Метод Зейделя')
print(all_results[3])

Метод простой итерации
+--------+-----------------+------------------------+
|  eps   | кол-во итераций |       norm(x-x*)       |
+--------+-----------------+------------------------+
| 1e-10  |       110       |  4.75963848520387e-11  |
| 1e-09  |       100       |  5.49402752631449e-10  |
| 1e-08  |        91       | 4.979045043449642e-09  |
| 1e-07  |        82       | 4.488103403595847e-08  |
| 1e-06  |        72       | 5.180607170713733e-07  |
| 1e-05  |        63       | 4.695003690956021e-06  |
| 0.0001 |        53       | 5.4194316978649766e-05 |
| 0.001  |        44       | 0.0004885067038469872  |
|  0.01  |        35       |  0.004427165970552566  |
|  0.1   |        25       |   0.0511026700296068   |
+--------+-----------------+------------------------+
Метод Зейделя
+--------+-----------------+------------------------+
|  eps   | кол-во итераций |       norm(x-x*)       |
+--------+-----------------+------------------------+
| 1e-10  |        48       | 1.40184424391426

Матрица Гильберта 2х2

In [14]:
print('Метод простой итерации')
print(all_results[4])
print('Метод Зейделя')
print(all_results[5])

Метод простой итерации
+--------+-----------------+-----------------------+
|  eps   | кол-во итераций |       norm(x-x*)      |
+--------+-----------------+-----------------------+
| 1e-10  |       159       | 7.107057428807342e-11 |
| 1e-09  |       143       | 7.099023237467189e-10 |
| 1e-08  |       127       | 7.091013902858102e-09 |
| 1e-07  |       111       | 7.083016217195015e-08 |
| 1e-06  |        95       | 7.075027446240274e-07 |
| 1e-05  |        79       |  7.06704768752483e-06 |
| 0.0001 |        63       | 7.059076928121795e-05 |
| 0.001  |        47       | 0.0007051115158699372 |
|  0.01  |        31       |  0.007043162369159001 |
|  0.1   |        15       |   0.0703521854938583  |
+--------+-----------------+-----------------------+
Метод Зейделя
+--------+-----------------+------------------------+
|  eps   | кол-во итераций |       norm(x-x*)       |
+--------+-----------------+------------------------+
| 1e-10  |        77       | 2.881172683297252e-10  |
| 1e-

Матрица Гильберта 3х3

In [15]:
print('Метод простой итерации')
print(all_results[6])
print('Метод Зейделя')
print(all_results[7])

Метод простой итерации
+--------+-----------------+------------------------+
|  eps   | кол-во итераций |       norm(x-x*)       |
+--------+-----------------+------------------------+
| 1e-10  |       500       | 6.015256084320646e+117 |
| 1e-09  |       500       | 6.015256084320646e+117 |
| 1e-08  |       500       | 6.015256084320646e+117 |
| 1e-07  |       500       | 6.015256084320646e+117 |
| 1e-06  |       500       | 6.015256084320646e+117 |
| 1e-05  |       500       | 6.015256084320646e+117 |
| 0.0001 |       500       | 6.015256084320646e+117 |
| 0.001  |       500       | 6.015256084320646e+117 |
|  0.01  |       500       | 6.015256084320646e+117 |
|  0.1   |       500       | 6.015256084320646e+117 |
+--------+-----------------+------------------------+
Метод Зейделя
+--------+-----------------+------------------------+
|  eps   | кол-во итераций |       norm(x-x*)       |
+--------+-----------------+------------------------+
| 1e-10  |       957       | 5.09380130719785

Матрица Гильберта 4х4

In [16]:
print('Метод простой итерации')
print(all_results[8])
print('Метод Зейделя')
print(all_results[9])

Метод простой итерации
+--------+-----------------+------------+
|  eps   | кол-во итераций | norm(x-x*) |
+--------+-----------------+------------+
| 1e-10  |       500       |    inf     |
| 1e-09  |       500       |    inf     |
| 1e-08  |       500       |    inf     |
| 1e-07  |       500       |    inf     |
| 1e-06  |       500       |    inf     |
| 1e-05  |       500       |    inf     |
| 0.0001 |       500       |    inf     |
| 0.001  |       500       |    inf     |
|  0.01  |       500       |    inf     |
|  0.1   |       500       |    inf     |
+--------+-----------------+------------+
Метод Зейделя
+--------+-----------------+------------------------+
|  eps   | кол-во итераций |       norm(x-x*)       |
+--------+-----------------+------------------------+
| 1e-10  |      14705      | 1.0288659788575158e-07 |
| 1e-09  |      12333      | 1.0287240021075273e-06 |
| 1e-08  |       9960      | 1.0295831946706332e-05 |
| 1e-07  |       7588      | 0.00010294435667159891